In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

In [ ]:
url = r'https://www.kaggle.com/competitions/plant-seedlings-classification/data'
od.download(url)

In [ ]:
images_path = r'/content/plant-seedlings-classification/train/*/*.png'

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import cv2
import imageio
import random
from glob import glob
import matplotlib.pyplot as plt    
import seaborn as sns
%matplotlib inline

In [ ]:
images = glob(images_path)

img_size = 128
train_images = []
train_labels = []
for i in images:
    train_images.append(cv2.resize(cv2.imread(i), (img_size, img_size))) 
    train_labels.append(i.split('/')[-2])
train_X = np.asarray(train_images)
train_Y = pd.DataFrame(train_labels)

In [ ]:
train_Y.rename(columns={0:'species'},inplace=True)
_, train_count = np.unique(train_Y,return_counts=True)
df = pd.DataFrame(data = train_count)
a = train_Y['species'].unique()
a = a.tolist()
a.sort()
df['Index'] = a
df.columns = ['Train','Name']
df

In [ ]:
plt.figure(figsize=(10,5))
chart = sns.countplot(
    data=train_Y,
    x='species'
)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)

In [ ]:
TRAIN_DIR = r'/content/plant-seedlings-classification/train'
CLASSES = [folder[len(TRAIN_DIR) + 1:] for folder in glob(TRAIN_DIR + '/*')]
CLASSES.sort()

TARGET_SIZE = (64, 64)
TARGET_DIMS = (64, 64, 3) # add channel for RGB
N_CLASSES = 42
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 64

In [ ]:
def plot_one_sample_of_each(base_path):
    cols = 4
    rows = int(np.ceil(len(CLASSES) / 3))
    fig = plt.figure(figsize=(16, 20))
    
    for i in range(len(CLASSES)):
        cls = CLASSES[i]
        img_path = base_path + '/' + cls + '/**'
        path_contents = glob(img_path)
    
        imgs = random.sample(path_contents, 1)

        sp = plt.subplot(rows, cols, i + 1)
        plt.imshow(imageio.imread(imgs[0]))
        plt.title(cls)
        sp.axis('off')

    plt.show()

In [ ]:
plot_one_sample_of_each(TRAIN_DIR)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
y = LabelBinarizer().fit_transform(train_Y.species)
train_label = np.array(y,dtype=np.float32)
train_label

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_X, train_label,test_size=0.3,random_state=7)

In [ ]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=180,  
        zoom_range = 0.1,
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=True  
    )  
datagen.fit(train_X)

In [ ]:
from os import listdir
from os.path import isfile, join
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten , Conv2D, MaxPool2D, BatchNormalization, MaxPooling2D
import matplotlib.pyplot as plt

In [ ]:
model0 = Sequential ([
(Conv2D(32, kernel_size=(3, 3),
            activation = 'relu',
            kernel_initializer='he_normal',
            input_shape=(128,128,3))),
MaxPooling2D((2,2)),(Dropout(0.25)),
Conv2D(64,
        kernel_size=(3,3),
        activation='relu'),
MaxPooling2D(pool_size=(2,2)),
Dropout(0.3),
Conv2D(128, (3,3), activation='relu'),
Dropout(0.40),
Flatten(),
Dense(128, activation='relu'),
Dropout(0.3),
Dense(12, activation = 'softmax')]) 

In [ ]:
model0.summary()

In [ ]:
#Saving the best model using model checkpoint callback
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = tf.keras.callbacks.ModelCheckpoint('plant_growth_classifier.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10)


In [ ]:
model0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics= ['accuracy'])
batch_size = 32
epochs = 30

In [ ]:
history = model0.fit(X_train, y_train ,batch_size=batch_size, epochs=epochs ,validation_data=(X_test, y_test),
           callbacks = [early_stopping,checkpoint],
                     verbose = 1)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
loss, acc = model0.evaluate(X_test,y_test)
loss1, acc1 = model0.evaluate(X_train,y_train)
print('Test loss:', loss,'   Test accuracy:', acc)
print('Train loss:', loss1,'   Train accuracy:',acc1)

In [ ]:
model0.save('plantgrowthmodel.h5')